In [18]:
# IMport Dependencies
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
from api_key import api_key


In [19]:
# Create an empty list for player data from all players from all states
all_states_data = []
# These columns will be used label the dataframe columns
columns = ['url', 'name', 'years', 'gp', 'mp', 'fg', 'fga', 'threep', 'threepa', 'ft', 'fta', 
           'orb', 'trb', 'ast', 'stl', 'blk', 'to', 'pf', 'pts', 'birthday', 'city', 'state', 'lat', 'lon']
# These are the data stat on the website that have the same format
data_stats = ['years', 'g', 'mp', 'fg', 'fga', 'fg3', 'fg3a', 'ft', 'fta', 
              'orb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'birth_date']

In [20]:
columns = ['url', 'name', 'years', 'gp', 'mp', 'fg', 'fga', 'threep', 'threepa', 'ft', 'fta', 
           'orb', 'trb', 'ast', 'stl', 'blk', 'to', 'pf', 'pts', 'birthday', 'city', 'state', 'lat', 'lon']

In [11]:
# In order to successfully compile the database, we must in one list of states at a time
# Alternate between commenting in/out each states list while reinitialize the variable 
# And running the for state in states loop
# Otherwise we risk a 429 Error for too many requests

states = ['AL','AK','AR','AZ','CA','CO','CT','DC','DE','FL','GA']
# states = ['HI','ID','IL','IN','IA','KS','KY','LA','ME','MD']
# states = ['MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ']
# states = ['NM','NY','NC','ND','OH','OK','OR','PA','RI','SC']
# states = ['SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

print(f'Ready to scrape the following states: {states}')

Ready to scrape the following states: ['SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']


In [12]:
# This loop iterates through the selected list of states
for state in states:
        
        url = f'https://www.basketball-reference.com/friv/birthplaces.fcgi?country=US&state={state}'
        html = urlopen(url)
        soup = BeautifulSoup(html, features='lxml')
        
        tbody = soup('tbody')
        

        for t in tbody:
                # Remove header breaks in website table
                for tr in t.select('.thead'):
                      tr.decompose()

                rows = t.findAll('tr')
                
                for row in rows:
                    # Create a list for each player's data
                    player_data = []
                    # The player name is in a hyperlink, so we must go into an a tag to get it
                    name = row.find('a').text
                    city = row.find('td', attrs={'data-stat': 'birth_city'}).text.replace(" ", "_")
                     # Url for API request
                    target_url = f"https://api.geoapify.com/v1/geocode/search?city={city}&state={state}&format=json&apiKey={api_key}"

                    # Make API request to get latitude and longitude
                    response = requests.get(target_url)
                    data = response.json()

                    # Extract latitude and longitude from the API response
                    if 'results' in data and data['results']:
                        lat = data['results'][0]['lat']
                        lon = data['results'][0]['lon']
                        #print(f'Coordinates for {city},{state}: lat: {lat}, lon: {lon}')

                    # Find the href extension for the player
                    player_url = row.find('a').get('href')
                    # Concatenate the base url for the website and the player extension
                    total_url = f'https://www.basketball-reference.com{player_url}'
                    # Each .append() adds player data to the list
                    player_data.append(total_url) 
                    player_data.append(name)
                    # This for loop allows us to gather many data-stat values
                    for stat in data_stats:
                        # This grabs the current data-stat from the list
                        data = row.find('td', attrs={'data-stat': stat}).text
                        player_data.append(data) 
                    player_data.append(city)
                    player_data.append(state)
                    player_data.append(lat)
                    player_data.append(lon)
                    # This appends the players data to the all_states_data list
                    all_states_data.append(player_data)
        print(f'Player data from {state} successfully scraped')

print('Done compiling, go to next list if needed')  

Player data from SD successfully scraped
Player data from TN successfully scraped
Player data from TX successfully scraped
Player data from UT successfully scraped
Player data from VT successfully scraped
Player data from VA successfully scraped
Player data from WA successfully scraped
Player data from WV successfully scraped
Player data from WI successfully scraped
Player data from WY successfully scraped
Done compiling, go to next list if needed


In [21]:
print(all_states_data)

[['https://www.basketball-reference.com/players/a/anslemi01.html', 'Michael Ansley', '3', '149', '2143', '383', '746', '0', '0', '260', '360', '311', '621', '67', '51', '24', '85', '284', '1026', 'Feb 8, 1967', 'Birmingham', 'AL', 33.5206824, -86.8024326], ['https://www.basketball-reference.com/players/a/askinke01.html', 'Keith Askins', '9', '486', '7983', '677', '1688', '275', '774', '223', '311', '525', '1428', '375', '278', '170', '321', '1094', '1852', 'Dec 15, 1967', 'Athens', 'AL', 34.8045487, -86.9711698], ['https://www.basketball-reference.com/players/b/baileca01.html', 'Carl Bailey', '1', '1', '7', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '2', '2', '2', 'Apr 23, 1958', 'Birmingham', 'AL', 33.5206824, -86.8024326], ['https://www.basketball-reference.com/players/b/barklch01.html', 'Charles Barkley', '16', '1073', '39330', '8435', '15605', '538', '2020', '6349', '8643', '4260', '12546', '4215', '1648', '888', '3376', '3287', '23757', 'Feb 20, 1963', 'Leeds', 'AL', 3

In [22]:
# Create Dataframe
all_states_df = pd.DataFrame(all_states_data)
all_states_df.columns = columns
all_states_df

,url,name,years,gp,mp,fg,fga,threep,threepa,ft,...,stl,blk,to,pf,pts,birthday,city,state,lat,lon
0,https://www.basketball-reference.com/players/a...,Michael Ansley,3,149,2143,383,746,0,0,260,...,51,24,85,284,1026,"Feb 8, 1967",Birmingham,AL,33.520682,-86.802433
1,https://www.basketball-reference.com/players/a...,Keith Askins,9,486,7983,677,1688,275,774,223,...,278,170,321,1094,1852,"Dec 15, 1967",Athens,AL,34.804549,-86.971170
2,https://www.basketball-reference.com/players/b...,Carl Bailey,1,1,7,1,1,0,0,0,...,0,0,2,2,2,"Apr 23, 1958",Birmingham,AL,33.520682,-86.802433
3,https://www.basketball-reference.com/players/b...,Charles Barkley,16,1073,39330,8435,15605,538,2020,6349,...,1648,888,3376,3287,23757,"Feb 20, 1963",Leeds,AL,33.543921,-86.542755
4,https://www.basketball-reference.com/players/b...,Eric Bledsoe,12,756,21021,3689,8169,854,2545,2116,...,1028,344,1888,1493,10348,"Dec 9, 1989",Birmingham,AL,33.520682,-86.802433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,https://www.basketball-reference.com/players/g...,Vern Gardner,3,151,507,514,1493,,,311,...,,,,445,1339,"May 14, 1925",Afton,WY,42.725277,-110.933402
4402,https://www.basketball-reference.com/players/j...,James Johnson,15,774,15524,2307,4870,381,1268,827,...,581,653,1044,1744,5822,"Feb 20, 1987",Cheyenne,WY,41.139981,-104.820246
4403,https://www.basketball-reference.com/players/p...,John Pilch,1,9,41,1,10,,,3,...,,,,10,5,"Jul 11, 1925",Sheridan,WY,44.797194,-106.956179
4404,https://www.basketball-reference.com/players/r...,Moe Radovich,1,4,33,5,13,,,4,...,,,,5,14,"May 5, 1929",Hot_Springs,WY,43.700875,-108.409568


In [23]:
# Check datatypes
all_states_df.dtypes

url          object
name         object
years        object
gp           object
mp           object
fg           object
fga          object
threep       object
threepa      object
ft           object
fta          object
orb          object
trb          object
ast          object
stl          object
blk          object
to           object
pf           object
pts          object
birthday     object
city         object
state        object
lat         float64
lon         float64
dtype: object

In [24]:
# Convert data strings to numbers
cols = ['years', 'gp', 'mp', 'fg', 'fga', 'threep', 'threepa', 'ft', 'fta', 
           'orb', 'trb', 'ast', 'stl', 'blk', 'to', 'pf', 'pts']
all_states_df[cols] = all_states_df[cols].apply(pd.to_numeric, errors='coerce')
all_states_df.dtypes

url          object
name         object
years         int64
gp            int64
mp          float64
fg            int64
fga           int64
threep      float64
threepa     float64
ft            int64
fta           int64
orb         float64
trb         float64
ast           int64
stl         float64
blk         float64
to          float64
pf            int64
pts           int64
birthday     object
city         object
state        object
lat         float64
lon         float64
dtype: object

In [33]:
##all_states_df = all_states_df.replace(to_replace="NaN",value=0)
##all_states_df = all_states_df.replace(to_replace="null",value=0)
all_states_df = all_states_df.fillna(0)


In [34]:
all_states_df

,url,name,years,gp,mp,fg,fga,threep,threepa,ft,...,stl,blk,to,pf,pts,birthday,city,state,lat,lon
0,https://www.basketball-reference.com/players/a...,Michael Ansley,3,149,2143.0,383,746,0.0,0.0,260,...,51.0,24.0,85.0,284,1026,"Feb 8, 1967",Birmingham,AL,33.520682,-86.802433
1,https://www.basketball-reference.com/players/a...,Keith Askins,9,486,7983.0,677,1688,275.0,774.0,223,...,278.0,170.0,321.0,1094,1852,"Dec 15, 1967",Athens,AL,34.804549,-86.971170
2,https://www.basketball-reference.com/players/b...,Carl Bailey,1,1,7.0,1,1,0.0,0.0,0,...,0.0,0.0,2.0,2,2,"Apr 23, 1958",Birmingham,AL,33.520682,-86.802433
3,https://www.basketball-reference.com/players/b...,Charles Barkley,16,1073,39330.0,8435,15605,538.0,2020.0,6349,...,1648.0,888.0,3376.0,3287,23757,"Feb 20, 1963",Leeds,AL,33.543921,-86.542755
4,https://www.basketball-reference.com/players/b...,Eric Bledsoe,12,756,21021.0,3689,8169,854.0,2545.0,2116,...,1028.0,344.0,1888.0,1493,10348,"Dec 9, 1989",Birmingham,AL,33.520682,-86.802433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,https://www.basketball-reference.com/players/g...,Vern Gardner,3,151,507.0,514,1493,0.0,0.0,311,...,0.0,0.0,0.0,445,1339,"May 14, 1925",Afton,WY,42.725277,-110.933402
4402,https://www.basketball-reference.com/players/j...,James Johnson,15,774,15524.0,2307,4870,381.0,1268.0,827,...,581.0,653.0,1044.0,1744,5822,"Feb 20, 1987",Cheyenne,WY,41.139981,-104.820246
4403,https://www.basketball-reference.com/players/p...,John Pilch,1,9,41.0,1,10,0.0,0.0,3,...,0.0,0.0,0.0,10,5,"Jul 11, 1925",Sheridan,WY,44.797194,-106.956179
4404,https://www.basketball-reference.com/players/r...,Moe Radovich,1,4,33.0,5,13,0.0,0.0,4,...,0.0,0.0,0.0,5,14,"May 5, 1929",Hot_Springs,WY,43.700875,-108.409568


In [35]:
# Create .json file to store data
out = all_states_df.to_json(orient='records')

with open('full_states_nba.json', 'w') as f:
    f.write(out)